In [14]:
import pandas as pd
import numpy as np

# Function to clean CSV files
def clean_csv(file_path):
    # Read CSV with correct delimiter
    df = pd.read_csv(file_path, sep=";", encoding="utf-8")

    # Convert numeric columns with comma (European format)
    for col in df.select_dtypes(include=["object"]).columns:
        if df[col].str.contains(",", regex=False).any():
            df[col] = df[col].str.replace(",", ".", regex=False)
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Convert date columns
    date_cols = ["start_date", "maturity_date", "reference_date"]
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")

    # Replace "null" with NaN
    df.replace("null", np.nan, inplace=True)

    # Remove duplicates
    df.drop_duplicates(inplace=True)

    return df

# Clean each dataset
accounts_df = clean_csv("Data Engineer Case Study - Accounts.csv")
deposits_df = clean_csv("Data Engineer Case Study - Deposits.csv")
loans_df = clean_csv("Data Engineer Case Study - Loans.csv")

# Save cleaned versions
accounts_df.to_csv("Cleaned_Accounts.csv", index=False)
deposits_df.to_csv("Cleaned_Deposits.csv", index=False)
loans_df.to_csv("Cleaned_Loans.csv", index=False)

print("Data cleaning completed! Cleaned files have been saved.")


Data cleaning completed! Cleaned files have been saved.


In [16]:
import pandas as pd
import numpy as np
import logging

# Setup logging for debugging
logging.basicConfig(filename="data_cleaning.log", level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Define expected data types
expected_dtypes = {
    "account_number": str,
    "amount": float,
    "account_name": str,
    "account_type": str,
    "reference_date": "datetime64",
    "customer": str,
    "customer_type": "category",
    "deposit_type": "category",
    "loan_type": "category",
    "country": "category",
    "currency": "category",
    "exchange_rate": float,
    "start_date": "datetime64",
    "maturity_date": "datetime64",
    "reference_date": "datetime64"
}

# Function to clean CSV files
def clean_csv(file_path, file_name):
    try:
        # Read CSV with correct delimiter
        df = pd.read_csv(file_path, sep=";", encoding="utf-8")

        # Standardize column names (strip spaces, lowercase)
        df.columns = df.columns.str.strip().str.lower()

        # Convert numeric columns (European format fix)
        for col in df.select_dtypes(include=["object"]).columns:
            if df[col].str.contains(",", regex=False).any():
                df[col] = df[col].str.replace(",", ".", regex=False)
                df[col] = pd.to_numeric(df[col], errors="coerce")

        # Convert date columns
        date_cols = ["start_date", "maturity_date", "reference_date"]
        for col in date_cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors="coerce")

        # Replace "null" with NaN
        df.replace("null", np.nan, inplace=True)

        # Handle categorical columns (convert to lowercase & strip spaces)
        for col in df.select_dtypes(include=["object"]).columns:
            df[col] = df[col].str.lower().str.strip()

        # Remove duplicates
        df.drop_duplicates(inplace=True)

        # Enforce expected data types
        for col, dtype in expected_dtypes.items():
            if col in df.columns:
                df[col] = df[col].astype(dtype, errors="ignore")

        # Handle negative and extreme values in amount
        if "amount" in df.columns:
            negative_values = df[df["amount"] < 0]
            if not negative_values.empty:
                logging.warning(f"Negative values found in {file_name}: \n{negative_values}")
                df["amount"] = df["amount"].apply(lambda x: np.nan if x < 0 else x)

        # Save cleaned file
        cleaned_file = f"Cleaned_{file_name}"
        df.to_csv(cleaned_file, index=False)

        logging.info(f"Successfully cleaned {file_name}")
        print(f"Data cleaning completed for {file_name}! Cleaned file saved as {cleaned_file}.")

        return df

    except Exception as e:
        logging.error(f"Error cleaning {file_name}: {str(e)}")
        print(f"Error processing {file_name}. Check logs for details.")
        return None

# Clean each dataset
accounts_df = clean_csv("Data Engineer Case Study - Accounts.csv", "Accounts.csv")
deposits_df = clean_csv("Data Engineer Case Study - Deposits.csv", "Deposits.csv")
loans_df = clean_csv("Data Engineer Case Study - Loans.csv", "Loans.csv")


Data cleaning completed for Accounts.csv! Cleaned file saved as Cleaned_Accounts.csv.
Data cleaning completed for Deposits.csv! Cleaned file saved as Cleaned_Deposits.csv.
Data cleaning completed for Loans.csv! Cleaned file saved as Cleaned_Loans.csv.
